In [10]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Lấy dữ liệu

In [67]:
def train_test_animals():
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    # Số lượng ảnh mỗi lớp cho tập huấn luyện và tập kiểm tra
    images_per_class_train = 750
    images_per_class_test = 250

    # Tạo ImageDataGenerator
    train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.25)

    # Tạo tập huấn luyện
    train_generator = train_datagen.flow_from_directory(
        'D:/Code/python/mat3533/practice08/data/CNN_MultiClass_data/animals',
        target_size=(150, 150),
        batch_size=images_per_class_train,  # Lấy hết ảnh trong mỗi lớp cho tập huấn luyện
        class_mode='categorical',
        subset='training',
        shuffle=True  # Xáo trộn dữ liệu
    )

    # Tạo tập kiểm tra
    test_generator = train_datagen.flow_from_directory(
        'D:/Code/python/mat3533/practice08/data/CNN_MultiClass_data/validation',
        target_size=(150, 150),
        batch_size=images_per_class_test,  # Lấy hết ảnh trong mỗi lớp cho tập kiểm tra
        class_mode='categorical',
        subset='validation',
        shuffle=True  # Xáo trộn dữ liệu
    )

    # Lấy ảnh và nhãn từ tập dữ liệu
    train_images, labels_train = next(train_generator)
    test_images, labels_test = next(test_generator)

    return train_images, test_images, labels_train, labels_test

X_train, X_test, y_train, y_test = train_test_animals()

Found 2250 images belonging to 3 classes.
Found 750 images belonging to 3 classes.


Chuẩn bị dữ liệu train, test để phân loại 2 lớp chó, mèo

In [68]:
def get_data(images, labels, class_indices, num_images):
    filtered_images = []
    filtered_labels = []
    for i, label in enumerate(labels):
        if label[class_indices[0]] == 1 or label[class_indices[1]] == 1:
            filtered_images.append(images[i])
            if label[class_indices[0]] == 1:
                filtered_labels.append(0)  # Nhãn cho lớp 0
            else:
                filtered_labels.append(1)  # Nhãn cho lớp 1
            num_images -= 1
            if num_images == 0:
                break
    return np.array(filtered_images), np.array(filtered_labels)

class_indices = [0, 1]

X_train, y_train = get_data(X_train, y_train, class_indices, 150)

X_test, y_test = get_data(X_test, y_test, class_indices, 100)

Khởi tạo mô hình CNN

In [70]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # 2 lớp đầu ra với softmax activation
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


Huấn luyện và chạy dự đoán mô hình CNN

In [73]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

predictions = model.predict(X_test)

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 235ms/step - accuracy: 1.0000 - loss: 0.0146 - val_accuracy: 0.5000 - val_loss: 2.5000
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 231ms/step - accuracy: 0.9869 - loss: 0.0286 - val_accuracy: 0.5100 - val_loss: 4.0727
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 218ms/step - accuracy: 1.0000 - loss: 0.0041 - val_accuracy: 0.5600 - val_loss: 2.8132
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 232ms/step - accuracy: 1.0000 - loss: 0.0054 - val_accuracy: 0.5000 - val_loss: 3.1321
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 283ms/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 0.5100 - val_loss: 3.7666
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 267ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 0.5100 - val_loss: 3.8586
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 256ms/step - accuracy: 1.0000 - loss: 8.8923e-04 - val_accuracy: 0.5000 - val_loss: 3.4932
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 244ms/step - accuracy: 1.0000 - loss: 6.1262e-04 - val_accuracy: 0.5100 - v